# Notebook 05: Advanced Workflows & Model Fitting

This is where we tackle more sophisticated image-processing scenarios.

**In this notebook, you will:**
- Generate images using parametric data sources (e.g., 2D Gaussian generator)
- Play through image stacks with a stack player
- Explore model fitting with `PolynomialFittingModel`
- Implement `ModelFittingContextOperation`
- Combine feature extraction and fitting in a multi-node pipeline

You’ll see how to build end-to-end systems that generate data, analyze features, and fit models—all within Semantiva’s flexible architecture.

---

1. **Parametric Data Sources**
    
    - `ParametricImageStackGenerator`: 2D Gaussian generator
    - Viewing parametric stacks (image stack player)
2. **Fitting Models**
    
    - PolynomialFittingModel
    - ModelFittingContextOperation
3. **End-to-End Feature Extraction & Fitting**
    
    - Designing a multi-node pipeline (`ProbeNode` + `ContextOperationNode`)
    - Combining parametric data generation, feature extraction, and fitting

> **Goal**: Illustrate sophisticated data generation and fitting scenarios, equipping users with tools for advanced image analysis projects.

In [1]:
from semantiva.logger import Logger
from semantiva.specializations.image.image_probes import (
    TwoDGaussianFitterProbe,
    TwoDTiltedGaussianFitterProbe,
)
from semantiva.workflows.fitting_model import PolynomialFittingModel
from semantiva.context_operations.context_operations import ModelFittingContextOperation
from semantiva.payload_operations.pipeline import Pipeline

from semantiva.specializations.image.image_loaders_savers_generators import (
    TwoDGaussianImageGenerator,
    ParametricImageStackGenerator,
)


## Generate an image stack with parametric 2d gaussian signal
generator = ParametricImageStackGenerator(
    num_frames=10,
    parametric_expressions={
        "center": "(350 + 200 * t, 425 - 100 * t + 100  * t ** 2)",  # Time-dependent center position
        "std_dev": "(50 + 20 * t, 20)",  # Standard deviation increases linearly over frames
        "amplitude": "100",  # Fixed amplitude for all frames
        "angle": "60 + 5 * t",
    },
    param_ranges={
        "t": (-1, 2)
    },  # Time variable range for controlling the parametric evolution
    image_generator=TwoDGaussianImageGenerator(),
    image_generator_params={"image_size": (1024, 1024)},  # Image resolution
)

# Retrieve the generated image stack
image_stack = generator.get_data()

In [2]:
from semantiva.specializations.image.image_viewers import ImageStackAnimator


ImageStackAnimator.view(image_stack=image_stack)

In [3]:
# Use the generator's parametric 't_values' as the independent variable for model fitting operations.
t_values = generator.t_values

# Create a sample context dictionary containing independent variable values.
# These t_values will later be used by the model fitting operations.
context_dict = {"t_values": t_values}

# --- Define Pipeline Configuration for Feature Extraction and Model Fitting ---
# This pipeline is configured to:
# 1. Extract fitting parameters from a 2D Gaussian fitter probe.
# 2. Use linear (1st degree polynomial) fits to model specific features extracted from the probe.
node_configurations = [
    {
        "operation": TwoDTiltedGaussianFitterProbe,
        # Extract complete fitting parameters from a 2D Gaussian fitter probe.
        # This node isolates parameters (e.g., standard deviation, angle) from the model fitter.
        "context_keyword": "gaussian_fit_parameters",
        # The extracted parameters will be stored in the context under the keyword 'gaussian_fit_parameters'.
    },
    {
        "operation": ModelFittingContextOperation,
        "parameters": {
            "fitting_model": PolynomialFittingModel(degree=1),
            # Use a linear model to fit the data.
            "independent_var_key": "t_values",
            # The independent variable is taken from the 't_values' in the context.
            "dependent_var_key": (
                "gaussian_fit_parameters",
                "std_dev_x",
            ),
            # Extracts the x-component of the standard deviation from the Gaussian fit parameters.
            "context_keyword": "std_dev_coefficients",
            # The resulting linear fit coefficients for std_dev_x will be saved under 'std_dev_coefficients'.
        },
    },
    {
        "operation": ModelFittingContextOperation,
        "parameters": {
            "fitting_model": PolynomialFittingModel(degree=1),
            # Use a linear model to fit the orientation feature.
            "independent_var_key": "t_values",
            "dependent_var_key": (
                "gaussian_fit_parameters",
                "angle",
            ),
            # Extracts the orientation angle from the Gaussian fit parameters.
            "context_keyword": "orientation_coefficients",
            # The resulting linear fit coefficients for the angle will be stored under 'orientation_coefficients'.
        },
    },
]

# --- Initialize Pipeline ---
# Create a pipeline with the above node configurations.
# This pipeline will first extract the fitting parameters from the Gaussian fitter probe,
# then perform linear fits on both the standard deviation and orientation data.
pipeline = Pipeline(node_configurations)

2025-02-22 17:11:23,937 - INFO     - Initialized Node 1: ProbeContextInjectorNode (pipeline)
2025-02-22 17:11:23,939 - INFO     - Initializing ModelFittingContextOperation (context_operations)
2025-02-22 17:11:23,939 - INFO     - Initialized Node 2: ContextNode (pipeline)
2025-02-22 17:11:23,940 - INFO     - Initializing ModelFittingContextOperation (context_operations)
2025-02-22 17:11:23,941 - INFO     - Initialized Node 3: ContextNode (pipeline)
2025-02-22 17:11:23,942 - INFO     - Initialized Pipeline (pipeline)


In [4]:
# --- Define Sample Context Dictionary ---
# Create a context dictionary that provides independent variable values required for model fitting.
# In this example, 't_values' is used as the independent variable for the linear fits.
context_dict = {"t_values": t_values}  # Sample independent variable values

# --- Execute the Pipeline ---
# Process the provided image stack along with the context dictionary.
# The pipeline applies feature extraction from the Gaussian fitter probe and then fits models to the extracted parameters.
output_data, output_context = pipeline.process(image_stack, context_dict)

# --- Display Pipeline Execution Results ---
# Retrieve and display the linear fitting coefficients for the x-component of the standard deviation.
# These coefficients were stored in the output context under the key 'std_dev_coefficients'.
print("Fitting Results for std_dev_x:", output_context.get_value("std_dev_coefficients"))

# Retrieve and display the linear fitting coefficients for the orientation feature.
# These coefficients were stored in the output context under the key 'orientation_coefficients'.
print("Fitting Results for orientation:", output_context.get_value("orientation_coefficients"))

2025-02-22 17:11:23,964 - INFO     - Start processing pipeline (pipeline)
2025-02-22 17:11:45,585 - INFO     - Pipeline execution complete. (pipeline)
Fitting Results for std_dev_x: {'coeff_0': 49.99999999999995, 'coeff_1': 20.000000000000014}
Fitting Results for orientation: {'coeff_0': 59.999999999999986, 'coeff_1': 5.000000000000016}


In [5]:
# --- Enable Debug Logging ---
# Set the logger level to DEBUG to enable detailed logging for troubleshooting.
# This will output verbose diagnostic information during the pipeline execution.
Logger(level="DEBUG")

# --- Initialize Pipeline ---
# Create a pipeline with the above node configurations.
# This pipeline will first extract the fitting parameters from the Gaussian fitter probe,
# then perform linear fits on both the standard deviation and orientation data.
pipeline = Pipeline(node_configurations)
print("=====================================================")
print("Pipeline initialized with debug logging enabled.")


# --- Execute the Pipeline with Debug Logging ---
# Process the image stack using the defined pipeline along with the context dictionary.
# The context dictionary supplies the independent variable ('t_values') for model fitting.
output_data, output_context = pipeline.process(image_stack, context_dict)
print("=====================================================")
print("Pipeline execution completed with debug logging enabled.")

# --- Display Pipeline Execution Results ---
# Print the linear fit coefficients for the x-component of the standard deviation,
# which were stored in the output context under the key 'std_dev_coefficients'.
print("Fitting Results for std_dev_x:", output_context.get_value("std_dev_coefficients"))

# Print the linear fit coefficients for the orientation feature,
# which were stored in the output context under the key 'orientation_coefficients'.
print("Fitting Results for orientation:", output_context.get_value("orientation_coefficients"))

2025-02-22 17:11:45,593 - INFO     - Logger verbosity level changed from INFO to DEBUG. (logger)
2025-02-22 17:11:45,594 - DEBUG    - Initializing ProbeContextInjectorNode (TwoDTiltedGaussianFitterProbe) (nodes)
2025-02-22 17:11:45,595 - INFO     - Initialized Node 1: ProbeContextInjectorNode (pipeline)
2025-02-22 17:11:45,595 - DEBUG    - Initializing ContextNode (ModelFittingContextOperation) (nodes)
2025-02-22 17:11:45,596 - INFO     - Initializing ModelFittingContextOperation (context_operations)
2025-02-22 17:11:45,596 - INFO     - Initialized Node 2: ContextNode (pipeline)
2025-02-22 17:11:45,596 - DEBUG    - Initializing ContextNode (ModelFittingContextOperation) (nodes)
2025-02-22 17:11:45,597 - INFO     - Initializing ModelFittingContextOperation (context_operations)
2025-02-22 17:11:45,597 - INFO     - Initialized Node 3: ContextNode (pipeline)
2025-02-22 17:11:45,598 - INFO     - Initialized Pipeline (pipeline)
2025-02-22 17:11:45,599 - DEBUG    - Pipeline Structure:
	Requir